In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import unidecode
import emoji
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

import nltk.corpus
#nltk.download("stopwords")
from nltk.corpus import stopwords

In [2]:
tweets = pd.read_excel("./data/authored_politician_tweets_10_19.xlsx")

In [3]:
tweets.head()

,id,tweet_url,created_at,parsed_created_at,user_screen_name,text,tweet_type,coordinates,hashtags,media,...,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_statuses_count,user_time_zone,user_urls,user_verified
0,1581019560849072128,https://twitter.com/golden4congress/status/158...,Fri Oct 14 20:30:11 +0000 2022,2022-10-14T20:30:11+00:00,golden4congress,We’ve got some merch so that you (or your pup)...,original,NaN,Goldens4Golden,https://pbs.twimg.com/media/FfDouM-X0AEIvdy.jpg,...,203,8478,179,279,"Maine, USA",Jared Golden for Congress,630,NaN,http://www.JaredGoldenForCongress.com,True
1,1580959199152205825,https://twitter.com/golden4congress/status/158...,Fri Oct 14 16:30:20 +0000 2022,2022-10-14T16:30:20+00:00,golden4congress,Dan is a farmer in The County. He’s also a Rep...,original,NaN,mepolitics,https://pbs.twimg.com/ext_tw_video_thumb/15809...,...,203,8478,179,279,"Maine, USA",Jared Golden for Congress,630,NaN,http://www.JaredGoldenForCongress.com,True
2,1580902528014614528,https://twitter.com/golden4congress/status/158...,Fri Oct 14 12:45:09 +0000 2022,2022-10-14T12:45:09+00:00,golden4congress,Jose knows Mainers can count on Jared to have ...,original,NaN,mepolitics,https://pbs.twimg.com/ext_tw_video_thumb/15809...,...,203,8478,179,279,"Maine, USA",Jared Golden for Congress,630,NaN,http://www.JaredGoldenForCongress.com,True
3,1580571765402923008,https://twitter.com/golden4congress/status/158...,Thu Oct 13 14:50:49 +0000 2022,2022-10-13T14:50:49+00:00,golden4congress,Mike is a retired fire chief and veteran and h...,original,NaN,NaN,https://pbs.twimg.com/ext_tw_video_thumb/15805...,...,203,8478,179,279,"Maine, USA",Jared Golden for Congress,630,NaN,http://www.JaredGoldenForCongress.com,True
4,1579921447627083776,https://twitter.com/golden4congress/status/157...,Tue Oct 11 19:46:41 +0000 2022,2022-10-11T19:46:41+00:00,golden4congress,We need to close the revolving door between Co...,original,NaN,mepolitics,https://pbs.twimg.com/media/Fe0CB77WYBERmUz.jpg,...,203,8478,179,279,"Maine, USA",Jared Golden for Congress,630,NaN,http://www.JaredGoldenForCongress.com,True


In [4]:
tweets['clean_text'] = tweets['text'].apply(lambda x: emoji.demojize(x))
tweets['clean_text'] = tweets['clean_text'].apply(lambda x: unidecode.unidecode(x))
tweets['clean_text'] = tweets['clean_text'].str.replace('\d+', '')
tweets['clean_text'] = tweets['clean_text'].apply(lambda s :s.translate(str.maketrans('', '', string.punctuation)))

<ipython-input-4-affea37685a7>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets['clean_text'] = tweets['clean_text'].str.replace('\d+', '')


In [5]:
stop_words = stopwords.words('english')
tweets['clean_stop'] = tweets['clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [6]:
corpus = list(tweets['clean_text'])
vectorizer = TfidfVectorizer(max_features=3000, ngram_range = (1,2))
X = vectorizer.fit_transform(corpus)

In [7]:
tf_idf_df = pd.DataFrame(X.todense(), columns = vectorizer.get_feature_names())

In [8]:
tf_idf_df = pd.concat([tf_idf_df,tweets[['user_screen_name','parsed_created_at']]], axis = 1)

In [9]:
climate_df = tf_idf_df.loc[(tf_idf_df['climate change'] > 0) | (tf_idf_df['climate'] > 0) | (tf_idf_df['environment'] > 0)]

In [10]:
climate_df[['climate change','climate', 'environment','user_screen_name','parsed_created_at']].sort_values('parsed_created_at', ascending=False).head(10)

,climate change,climate,environment,user_screen_name,parsed_created_at
839,0.323145,0.291592,0.180142,YadiraCaraveo,2022-10-14T20:23:10+00:00
966,0.155214,0.140059,0.000000,AngieCraigMN,2022-10-14T19:40:09+00:00
1113,0.000000,0.206415,0.000000,RepHerrell,2022-10-14T15:48:48+00:00
244,0.305053,0.275267,0.000000,RepValDemings,2022-10-13T17:08:20+00:00
977,0.176015,0.158829,0.000000,AngieCraigMN,2022-10-13T14:10:29+00:00
1070,0.000000,0.156093,0.000000,SusieLeeNV,2022-10-12T17:06:01+00:00
1111,0.000000,0.158921,0.000000,Gabe_NM,2022-10-09T21:34:37+00:00
816,0.000000,0.188025,0.000000,RepOHalleran,2022-10-09T18:45:00+00:00
1081,0.189970,0.171420,0.000000,SusieLeeNV,2022-10-08T19:34:29+00:00
4439,0.000000,0.147082,0.000000,SusieLeeNV,2022-10-07T18:18:18+00:00


In [11]:
climate_df.groupby(['user_screen_name']).agg({'parsed_created_at':'count'}).sort_values('parsed_created_at', ascending=False).rename(columns={'parsed_created_at':'Num Tweets'})

,Num Tweets
user_screen_name,
ERICSORENSEN,138
TheOtherMandela,84
Gabe_NM,80
RepSusieLee,74
SenCortezMasto,70
RepValDemings,66
wileynickel,55
SenatorHassan,55
RepOHalleran,49


In [12]:
converg_df = tf_idf_df[['student','loan','debt','relief','climate','change','environment']]

In [13]:
converg_df.corr()

,student,loan,debt,relief,climate,change,environment
student,1.000000,0.317871,0.245636,0.040468,-0.002661,0.002128,-0.002773
loan,0.317871,1.000000,0.153628,0.049247,-0.003872,-0.002559,-0.002040
debt,0.245636,0.153628,1.000000,0.051893,-0.004469,-0.001467,-0.002870
relief,0.040468,0.049247,0.051893,1.000000,-0.006514,-0.011825,-0.001024
climate,-0.002661,-0.003872,-0.004469,-0.006514,1.000000,0.376220,0.053580
change,0.002128,-0.002559,-0.001467,-0.011825,0.376220,1.000000,0.022809
environment,-0.002773,-0.002040,-0.002870,-0.001024,0.053580,0.022809,1.000000


In [14]:
vectorizer = CountVectorizer(max_features=3000, ngram_range = (1,2))
doc_term = vectorizer.fit_transform(corpus)

In [15]:
dtm = pd.DataFrame(doc_term.todense(), columns = vectorizer.get_feature_names())

In [16]:
dtm.head()

,ability,ability to,able,able to,abortion,about,about how,about it,about my,about our,...,your family,your help,your service,your support,youre,yours,yourself,youth,youve,zero
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
